In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [3]:
postulantes = None
avisos = None 
postulaciones = None
test = None

In [3]:
test = pd.concat([test,postulaciones])
test = test.drop_duplicates(['idaviso','idpostulante'])
postulaciones = test[100000:].drop('id', axis=1)
test = test[:100000].drop('id', axis=1)

In [4]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,1.112295e+09,KBe4pdq,0.0,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,75.0,1.0,NaN,26.095588,177374,29.0,FEM,5
1,1.112477e+09,KBe4pdq,0.0,Full-time,Otro,Programación,Huenei IT Services,172.0,NaN,NaN,NaN,NaN,NaN,177374,29.0,FEM,5
2,1.112249e+09,KBe4pdq,0.0,Full-time,Senior / Semi-Senior,Recepcionista,HOTEL EUROPA,59.0,289.0,218.0,4.0,FEM,29.059406,177374,29.0,FEM,5
3,1.111704e+09,KBe4pdq,0.0,Part-time,Otro,Atención al Cliente,ADT Security Services S.A.,205.0,NaN,NaN,NaN,NaN,NaN,177374,29.0,FEM,5
4,1.112325e+09,KBe4pdq,0.0,Part-time,Senior / Semi-Senior,Ventas,TRABAJO ONLINE,142.0,33.0,22.0,0.0,FEM,29.109091,177374,29.0,FEM,5


In [5]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,739260.0,6M9ZQR,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,NaN,NaN,NaN,69266,42.0,FEM,21
1,739260.0,6v1xdL,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,NaN,NaN,NaN,83910,31.0,MASC,9
2,739260.0,ezRKm9,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,NaN,NaN,NaN,351736,36.0,FEM,12
3,758580.0,1Q35ej,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,NaN,NaN,NaN,NaN,NaN,15447,69.0,MASC,15
4,758580.0,EAN4J6,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,NaN,NaN,NaN,NaN,NaN,138590,32.0,FEM,11


In [6]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,NaN,NaN,NaN,69266,42.0,FEM,21
1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,NaN,NaN,NaN,83910,31.0,MASC,9
2,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,NaN,NaN,NaN,351736,36.0,FEM,12
3,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,NaN,NaN,NaN,NaN,NaN,15447,69.0,MASC,15
4,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,NaN,NaN,NaN,NaN,NaN,138590,32.0,FEM,11


In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    print(col)
    postulaciones[col] = postulaciones[col].fillna('na')
    test[col] = test[col].fillna('na')
    le.fit(pd.concat([postulaciones[col],test[col]]))
    test[col] = le.transform(test[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
TendenciaSexo
sexo


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,2,2,2,1272,178.0,61.0,75.0,1.0,2,26.095588,177374,29.0,1,5
1,1,3,145,2025,172.0,NaN,NaN,NaN,2,NaN,177374,29.0,1,5
2,1,4,153,1965,59.0,289.0,218.0,4.0,0,29.059406,177374,29.0,1,5
3,2,3,14,0,205.0,NaN,NaN,NaN,2,NaN,177374,29.0,1,5
4,2,4,185,3798,142.0,33.0,22.0,0.0,0,29.109091,177374,29.0,1,5


In [8]:
postulaciones.sample(5)
from sklearn.utils import shuffle
postulaciones = shuffle(postulaciones)

In [9]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [10]:
aviso = None
postulantes = None
postulaciones.fillna(-1,inplace=True)
test.fillna(-1,inplace=True)
test.drop([6],axis=1,inplace=True)
postulaciones.drop([6],axis=1,inplace=True)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=50 ,n_jobs=-1, algorithm='auto')
neigh.fit(postulaciones,resultado)
knn_result = neigh.predict_proba(test)

In [12]:
knn_result

array([[0.64, 0.36],
       [0.76, 0.24],
       [0.58, 0.42],
       ...,
       [0.56, 0.44],
       [0.58, 0.42],
       [0.78, 0.22]])

In [13]:
prob_1 = []
for x in knn_result:
    prob_1.append(x[0])
prob_1

[0.64,
 0.76,
 0.58,
 0.58,
 0.66,
 0.56,
 0.68,
 0.62,
 0.66,
 0.62,
 0.74,
 0.66,
 0.7,
 0.78,
 0.68,
 0.64,
 0.64,
 0.7,
 0.62,
 0.66,
 0.66,
 0.68,
 0.68,
 0.62,
 0.64,
 0.72,
 0.72,
 0.72,
 0.64,
 0.66,
 0.7,
 0.72,
 0.56,
 0.76,
 0.56,
 0.72,
 0.6,
 0.6,
 0.58,
 0.7,
 0.72,
 0.68,
 0.68,
 0.6,
 0.68,
 0.62,
 0.66,
 0.68,
 0.76,
 0.58,
 0.54,
 0.6,
 0.6,
 0.74,
 0.64,
 0.76,
 0.7,
 0.74,
 0.78,
 0.76,
 0.6,
 0.7,
 0.54,
 0.66,
 0.7,
 0.58,
 0.68,
 0.68,
 0.68,
 0.58,
 0.72,
 0.76,
 0.7,
 0.78,
 0.7,
 0.64,
 0.54,
 0.78,
 0.58,
 0.62,
 0.62,
 0.62,
 0.6,
 0.7,
 0.7,
 0.64,
 0.64,
 0.74,
 0.6,
 0.7,
 0.58,
 0.5,
 0.66,
 0.82,
 0.62,
 0.7,
 0.72,
 0.58,
 0.66,
 0.6,
 0.66,
 0.68,
 0.66,
 0.52,
 0.58,
 0.62,
 0.7,
 0.66,
 0.72,
 0.58,
 0.84,
 0.62,
 0.68,
 0.64,
 0.58,
 0.54,
 0.66,
 0.5,
 0.66,
 0.78,
 0.68,
 0.66,
 0.76,
 0.62,
 0.6,
 0.52,
 0.72,
 0.68,
 0.66,
 0.74,
 0.66,
 0.72,
 0.68,
 0.68,
 0.64,
 0.68,
 0.74,
 0.7,
 0.7,
 0.72,
 0.78,
 0.68,
 0.66,
 0.66,
 0.62,
 0.72,
 0.68,

In [14]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = prob_1
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_random.csv', index=False)
